In [1]:
# How many images in each folder?
import os

# Walk through 10 percent data directory and list number of files
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/train'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/fried_rice'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/sushi'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/hamburger'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ramen'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/pizza'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ice_cream'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_curry'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/steak'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/grilled_salmon'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/c

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

target_shape = (224, 224)
batch_size = 8

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

train_data = train_datagen.flow_from_directory(directory=train_dir,
                                               target_size=target_shape,
                                               batch_size=batch_size,
                                               class_mode='categorical')

test_data = test_datagen.flow_from_directory(directory=test_dir,
                                               target_size=target_shape,
                                               batch_size=batch_size,
                                               class_mode='categorical')


Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [25]:
import datetime

def tensorboard_callback(dir_name, experiment_name):
    log_dir = f'{dir_name}/experiment_name{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}'
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    print(f"Saving TensorBoard log files to: {log_dir}")
    return tb_callback


In [14]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers


In [15]:
# Resnet 50 V2 feature vector
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

# Original: EfficientNetB0 feature vector (version 1)
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

# # New: EfficientNetB0 feature vector (version 2)
# efficientnet_url = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2"

In [22]:
def create_model(model_url, num_classes=10):

    feature_extractor = hub.KerasLayer(model_url, 
                                       trainable=False,
                                       name='feature_extractor_layer',
                                       input_shape=target_shape+(3,))
    
    inputs = tf.keras.layers.Input(shape=(target_shape+(3,)))
    x = feature_extractor(inputs)
    x = tf.keras.layers.Dense(num_classes, activation='softmax', name='classifier')(x)
    return tf.keras.Model(inputs=inputs, outputs=x)
model = create_model(resnet_url)
model.summary()
model.compile(loss='categorical_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
feature_extractor_layer (Ker (None, 2048)              23564800  
_________________________________________________________________
classifier (Dense)           (None, 10)                20490     
Total params: 23,585,290
Trainable params: 20,490
Non-trainable params: 23,564,800
_________________________________________________________________


In [26]:
resnet_history = model.fit(train_data, 
                           epochs=20,
                           steps_per_epoch=len(train_data),
                           validation_data=test_data,
                           validation_steps=len(test_data),
                           callbacks=[tensorboard_callback(dir_name='tf_hub', experiment_name='resnet50')]
                           )

2021-11-11 10:20:11.456003: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-11-11 10:20:11.456027: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-11-11 10:20:11.459279: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1611] Profiler found 1 GPUs
2021-11-11 10:20:11.462348: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory
2021-11-11 10:20:11.462959: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so'; dlerror: libcupti.so: cannot open shared object file: No such file or directory
2021-11-11 10:20:11.462989: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1661] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be 

Saving TensorBoard log files to: tf_hub/experiment_name20211111-102011
Epoch 1/20
 1/94 [..............................] - ETA: 3:50 - loss: 2.6778 - accuracy: 0.2500

2021-11-11 10:20:14.469987: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-11-11 10:20:14.470009: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-11-11 10:20:14.470291: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1661] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.


 2/94 [..............................] - ETA: 1:03 - loss: 2.7704 - accuracy: 0.2500

2021-11-11 10:20:14.734379: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-11-11 10:20:14.734481: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1752] function cupti_interface_->Finalize()failed with error CUPTI could not be loaded or symbol could not be found.
2021-11-11 10:20:14.736082: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 0 callback api events and 0 activity events. 
2021-11-11 10:20:14.740637: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-11-11 10:20:14.749760: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: tf_hub/experiment_name20211111-102011/train/plugins/profile/2021_11_11_10_20_14
2021-11-11 10:20:14.752310: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to tf_hub/experiment_name20211111-102011/train/plugins/profile/2021_11_11_10_20_14/viktor-

94/94 [==============================] - 99s 1s/step - loss: 1.5150 - accuracy: 0.5147 - val_loss: 0.8440 - val_accuracy: 0.7352
Epoch 2/20
94/94 [==============================] - 99s 1s/step - loss: 0.6001 - accuracy: 0.8120 - val_loss: 0.7239 - val_accuracy: 0.7676
Epoch 3/20
94/94 [==============================] - 105s 1s/step - loss: 0.3772 - accuracy: 0.9093 - val_loss: 0.6498 - val_accuracy: 0.7924
Epoch 4/20
94/94 [==============================] - 110s 1s/step - loss: 0.2529 - accuracy: 0.9533 - val_loss: 0.6521 - val_accuracy: 0.7840
Epoch 5/20
94/94 [==============================] - ETA: 0s - loss: 0.1792 - accuracy: 0.9787

KeyboardInterrupt: 